<a href="https://colab.research.google.com/github/ChosenByFate/DL_Course_SamU/blob/main/04/assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Лабораторная работа 4

Tensorflow 2.x

1) Подготовка данных

2) Использование Keras Model API

3) Использование Keras Sequential + Functional API

https://www.tensorflow.org/tutorials

Для выполнения лабораторной работы необходимо установить tensorflow версии 2.0 или выше .

Рекомендуется использовать возможности Colab'а по обучению моделей на GPU.



In [1]:
import os
import tensorflow as tf
import numpy as np
import math
import timeit
import matplotlib.pyplot as plt

%matplotlib inline

# Подготовка данных
Загрузите набор данных из предыдущей лабораторной работы. 

In [2]:
def load_mnist(num_training=50000, num_validation=10000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    mnist = tf.keras.datasets.mnist.load_data()
    (X_train, y_train), (X_test, y_test) = mnist
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1], X_train.shape[2]))
    X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1], X_val.shape[2]))
    X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1], X_test.shape[2]))

    return X_train, y_train, X_val, y_val, X_test, y_test

# If there are errors with SSL downloading involving self-signed certificates,
# it may be that your Python version was recently installed on the current machine.
# See: https://github.com/tensorflow/tensorflow/issues/10779
# To fix, run the command: /Applications/Python\ 3.7/Install\ Certificates.command
#   ...replacing paths as necessary.

# Invoke the above function to get our data.
NHW = (0, 1, 2)
X_train, y_train, X_val, y_val, X_test, y_test = load_mnist()
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)

11493376/11490434 [==============================] - 0s 0us/step
Train data shape:  (50000, 1, 28, 28)
Train labels shape:  (50000,) int32
Validation data shape:  (10000, 1, 28, 28)
Validation labels shape:  (10000,)
Test data shape:  (10000, 1, 28, 28)
Test labels shape:  (10000,)


In [3]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(X_train, y_train, batch_size=64, shuffle=True)
val_dset = Dataset(X_val, y_val, batch_size=64, shuffle=False)
test_dset = Dataset(X_test, y_test, batch_size=64)

In [4]:
# We can iterate through a dataset like this:
for t, (x, y) in enumerate(train_dset):
    print(t, x.shape, y.shape)
    if t > 5: break

0 (64, 1, 28, 28) (64,)
1 (64, 1, 28, 28) (64,)
2 (64, 1, 28, 28) (64,)
3 (64, 1, 28, 28) (64,)
4 (64, 1, 28, 28) (64,)
5 (64, 1, 28, 28) (64,)
6 (64, 1, 28, 28) (64,)


#  Keras Model Subclassing API


Для реализации собственной модели с помощью Keras Model Subclassing API необходимо выполнить следующие шаги:

1) Определить новый класс, который является наследником tf.keras.Model.

2) В методе __init__() определить все необходимые слои из модуля tf.keras.layer

3) Реализовать прямой проход в методе call() на основе слоев, объявленных в __init__()

Ниже приведен пример использования keras API для определения двухслойной полносвязной сети. 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras

In [5]:
device = "GPU"

In [6]:
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации. 

Архитектура сети:
    
1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU 
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU 
5. Полносвязный слой 
6. Функция активации Softmax 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Conv2D

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dense

In [7]:
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(channel_1, kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=initializer)
        self.conv2 = tf.keras.layers.Conv2D(channel_2, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=initializer)
        self.fc3 = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        x = self.fc3(x)
        return x
        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [8]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    with tf.device(device):
        x = tf.zeros((64, 3, 32, 32))
        scores = model(x)
        print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


Пример реализации процесса обучения:

In [9]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during training
    """    
    with tf.device(device):

        
        loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()
        
        model = model_init_fn()
        optimizer = optimizer_init_fn()
        
        train_loss = tf.keras.metrics.Mean(name='train_loss')
        train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
    
        val_loss = tf.keras.metrics.Mean(name='val_loss')
        val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')
        
        t = 0
        for epoch in range(num_epochs):
            
            # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
            train_loss.reset_states()
            train_accuracy.reset_states()
            
            for x_np, y_np in train_dset:
                with tf.GradientTape() as tape:
                    
                    # Use the model function to build the forward pass.
                    scores = model(x_np, training=is_training)
                    loss = loss_fn(y_np, scores)
      
                    gradients = tape.gradient(loss, model.trainable_variables)
                    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
                    
                    # Update the metrics
                    train_loss.update_state(loss)
                    train_accuracy.update_state(y_np, scores)
                    
                    if t % print_every == 0:
                        val_loss.reset_states()
                        val_accuracy.reset_states()
                        for test_x, test_y in val_dset:
                            # During validation at end of epoch, training set to False
                            prediction = model(test_x, training=False)
                            t_loss = loss_fn(test_y, prediction)

                            val_loss.update_state(t_loss)
                            val_accuracy.update_state(test_y, prediction)
                        
                        template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                        print (template.format(t, epoch+1,
                                             train_loss.result(),
                                             train_accuracy.result()*100,
                                             val_loss.result(),
                                             val_accuracy.result()*100))
                    t += 1
        test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')
        for x_np, y_np in test_dset:
          prediction = model(x_np, training=False)
          test_accuracy.update_state(y_np, prediction)
        print("Test accuracy: {}".format(test_accuracy.result()*100))

In [10]:
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2
print_every = 100

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.96191143989563, Accuracy: 18.75, Val Loss: 2.659280776977539, Val Accuracy: 21.969999313354492
Iteration 100, Epoch 1, Loss: 0.6305063962936401, Accuracy: 80.81682586669922, Val Loss: 0.40173834562301636, Val Accuracy: 87.75
Iteration 200, Epoch 1, Loss: 0.5006923079490662, Accuracy: 84.7403564453125, Val Loss: 0.3152967393398285, Val Accuracy: 90.61000061035156
Iteration 300, Epoch 1, Loss: 0.4420437812805176, Accuracy: 86.61752319335938, Val Loss: 0.28252923488616943, Val Accuracy: 91.13999938964844
Iteration 400, Epoch 1, Loss: 0.4002740979194641, Accuracy: 87.98316955566406, Val Loss: 0.25382137298583984, Val Accuracy: 92.5
Iteration 500, Epoch 1, Loss: 0.3771345913410187, Accuracy: 88.70384216308594, Val Loss: 0.2375023514032364, Val Accuracy: 92.98999786376953
Iteration 600, Epoch 1, Loss: 0.35425707697868347, Accuracy: 89.37967681884766, Val Loss: 0.22392897307872772, Val Accuracy: 93.56999969482422
Iteration 700, Epoch 1, Loss: 0.3381066620349884, 

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9 . 

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50% .

In [11]:
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    model = ThreeLayerConvNet(channel_1=channel_1, channel_2=channel_2, num_classes=num_classes)
    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3089466094970703, Accuracy: 7.8125, Val Loss: 2.343623161315918, Val Accuracy: 9.710000038146973
Iteration 100, Epoch 1, Loss: 1.408502221107483, Accuracy: 55.863243103027344, Val Loss: 0.5576921105384827, Val Accuracy: 81.95000457763672
Iteration 200, Epoch 1, Loss: 0.9478008151054382, Accuracy: 70.70895385742188, Val Loss: 0.40607452392578125, Val Accuracy: 87.69000244140625
Iteration 300, Epoch 1, Loss: 0.7525426745414734, Accuracy: 76.9362564086914, Val Loss: 0.26948806643486023, Val Accuracy: 91.82999420166016
Iteration 400, Epoch 1, Loss: 0.6342098116874695, Accuracy: 80.564208984375, Val Loss: 0.24159488081932068, Val Accuracy: 92.5999984741211
Iteration 500, Epoch 1, Loss: 0.5629077553749084, Accuracy: 82.9091796875, Val Loss: 0.2175150066614151, Val Accuracy: 93.33000183105469
Iteration 600, Epoch 1, Loss: 0.5084498524665833, Accuracy: 84.5673828125, Val Loss: 0.2162906676530838, Val Accuracy: 93.30000305175781
Iteration 700, Epoch 1, Loss: 0.4672

# Использование Keras Sequential API для реализации последовательных моделей.

Пример для полносвязной сети:

In [12]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (28, 28, 1)
    hidden_layer_size, num_classes = 4000, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.0788803100585938, Accuracy: 9.375, Val Loss: 2.77116322517395, Val Accuracy: 10.260000228881836
Iteration 100, Epoch 1, Loss: 0.6452388167381287, Accuracy: 79.8731460571289, Val Loss: 0.41270115971565247, Val Accuracy: 87.9800033569336
Iteration 200, Epoch 1, Loss: 0.5119789242744446, Accuracy: 84.68594360351562, Val Loss: 0.3251451253890991, Val Accuracy: 90.94999694824219
Iteration 300, Epoch 1, Loss: 0.45307278633117676, Accuracy: 86.47737121582031, Val Loss: 0.2875993549823761, Val Accuracy: 91.44999694824219
Iteration 400, Epoch 1, Loss: 0.4085933566093445, Accuracy: 87.86627197265625, Val Loss: 0.2595386505126953, Val Accuracy: 92.41999816894531
Iteration 500, Epoch 1, Loss: 0.38435375690460205, Accuracy: 88.5978012084961, Val Loss: 0.2422180473804474, Val Accuracy: 93.16999816894531
Iteration 600, Epoch 1, Loss: 0.36104831099510193, Accuracy: 89.31208038330078, Val Loss: 0.23103424906730652, Val Accuracy: 93.66000366210938
Iteration 700, Epoch 1, Lo

Альтернативный менее гибкий способ обучения:

In [13]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.2201 - sparse_categorical_accuracy: 0.9366


[0.2201450616121292, 0.9366000294685364]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [14]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    input_shape = (28, 28, 1)
    channel_1, channel_2, num_classes = 12, 8, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)

    layers = [
        tf.keras.layers.Conv2D(channel_1, kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Conv2D(channel_2, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=initializer),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True) 
    pass

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 2.3700952529907227, Accuracy: 4.6875, Val Loss: 2.3460118770599365, Val Accuracy: 11.010000228881836
Iteration 100, Epoch 1, Loss: 2.2696645259857178, Accuracy: 16.475866317749023, Val Loss: 2.210325241088867, Val Accuracy: 22.75
Iteration 200, Epoch 1, Loss: 2.196300506591797, Accuracy: 22.916667938232422, Val Loss: 2.0295541286468506, Val Accuracy: 37.709999084472656
Iteration 300, Epoch 1, Loss: 2.1015801429748535, Accuracy: 28.929609298706055, Val Loss: 1.7250677347183228, Val Accuracy: 49.32999801635742
Iteration 400, Epoch 1, Loss: 1.9637075662612915, Accuracy: 34.86595916748047, Val Loss: 1.3170944452285767, Val Accuracy: 59.70000076293945
Iteration 500, Epoch 1, Loss: 1.8179776668548584, Accuracy: 39.85466766357422, Val Loss: 0.9881656765937805, Val Accuracy: 69.22000122070312
Iteration 600, Epoch 1, Loss: 1.6711903810501099, Accuracy: 44.969322204589844, Val Loss: 0.7792988419532776, Val Accuracy: 76.95000457763672
Iteration 700, Epoch 1, Loss: 1.54

In [15]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(X_train, y_train, batch_size=64, epochs=1, validation_data=(X_val, y_val))
model.evaluate(X_test, y_test)

313/313 [==============================] - 1s 2ms/step - loss: 0.3093 - sparse_categorical_accuracy: 0.9109


[0.30934759974479675, 0.9108999967575073]

# Использование Keras Functional API

Для реализации более сложных архитектур сети с несколькими входами/выходами, повторным использованием слоев, "остаточными" связями (residual connections) необходимо явно указать входные и выходные тензоры. 

Ниже представлен пример для полносвязной сети. 

In [16]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    with tf.device(device):
        scores = model(x)
        print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [17]:
input_shape = (28, 28, 1)
hidden_size, num_classes = 4000, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.2264785766601562, Accuracy: 7.8125, Val Loss: 2.9507534503936768, Val Accuracy: 18.420000076293945
Iteration 100, Epoch 1, Loss: 0.651204526424408, Accuracy: 80.5538330078125, Val Loss: 0.40293797850608826, Val Accuracy: 87.84000396728516
Iteration 200, Epoch 1, Loss: 0.5129328966140747, Accuracy: 84.87251281738281, Val Loss: 0.3190971612930298, Val Accuracy: 90.55000305175781
Iteration 300, Epoch 1, Loss: 0.4534941017627716, Accuracy: 86.61752319335938, Val Loss: 0.2998960018157959, Val Accuracy: 90.6500015258789
Iteration 400, Epoch 1, Loss: 0.40891700983047485, Accuracy: 87.9403076171875, Val Loss: 0.2581421434879303, Val Accuracy: 92.36000061035156
Iteration 500, Epoch 1, Loss: 0.3843814730644226, Accuracy: 88.65393829345703, Val Loss: 0.2416689395904541, Val Accuracy: 92.88999938964844
Iteration 600, Epoch 1, Loss: 0.36111631989479065, Accuracy: 89.3328857421875, Val Loss: 0.23612944781780243, Val Accuracy: 93.3499984741211
Iteration 700, Epoch 1, Los

Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут). 

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [18]:
class CustomConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, channel_3, channel_4, num_classes=10):
        super(CustomConvNet, self).__init__()
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.conv1 = tf.keras.layers.Conv2D(channel_1, kernel_size=(7,7), padding='same', activation='relu', kernel_initializer=initializer)
        self.bn1 = tf.keras.layers.BatchNormalization()
        self.conv2 = tf.keras.layers.Conv2D(channel_2, kernel_size=(5,5), padding='same', activation='relu', kernel_initializer=initializer)
        self.bn2 = tf.keras.layers.BatchNormalization()
        self.conv3 = tf.keras.layers.Conv2D(channel_3, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=initializer)
        self.bn3 = tf.keras.layers.BatchNormalization()
        self.conv4 = tf.keras.layers.Conv2D(channel_4, kernel_size=(3,3), padding='same', activation='relu', kernel_initializer=initializer)
        self.bn4 = tf.keras.layers.BatchNormalization()
        self.fc = tf.keras.layers.Dense(num_classes, activation='softmax', kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
    
    def call(self, input_tensor, training=False):
        ############################################################################
        # TODO: Construct a model that performs well on CIFAR-10                   #
        ############################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        x = self.conv1(input_tensor)
        x = self.bn1(x)
        x = self.conv2(x)
        x = self.bn2(x)
        x = self.conv3(x)
        x = self.bn3(x)
        x = self.conv4(x)
        x = self.bn4(x)
        x = self.flatten(x)
        x = self.fc(x)
        pass

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ############################################################################
        #                            END OF YOUR CODE                              #
        ############################################################################
        return x


print_every = 700
num_epochs = 10

# channel_1, channel_2, channel_3, channel_4 = 20, 16, 12, 10
# channel_1, channel_2, channel_3, channel_4 = 20, 18, 16, 14
# channel_1, channel_2, channel_3, channel_4 = 40, 34, 28, 24 # 98.4
channel_1, channel_2, channel_3, channel_4 = 100, 80, 60, 40 #98.73
# channel_1, channel_2, channel_3, channel_4 = 200, 140, 100, 40

# model = CustomConvNet(channel_1, channel_2, channel_3)

def model_init_fn():
    return CustomConvNet(channel_1, channel_2, channel_3, channel_4)

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate, epsilon=1e-08) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=num_epochs, is_training=True)

Iteration 0, Epoch 1, Loss: 2.727301597595215, Accuracy: 18.75, Val Loss: 2.2908267974853516, Val Accuracy: 14.329999923706055
Iteration 700, Epoch 1, Loss: 0.1598145067691803, Accuracy: 95.07845306396484, Val Loss: 0.08257576078176498, Val Accuracy: 97.65999603271484
Iteration 1400, Epoch 2, Loss: 0.06215738132596016, Accuracy: 98.07148742675781, Val Loss: 0.070359967648983, Val Accuracy: 97.90999603271484
Iteration 2100, Epoch 3, Loss: 0.04115518182516098, Accuracy: 98.66154479980469, Val Loss: 0.060202423483133316, Val Accuracy: 98.25999450683594
Iteration 2800, Epoch 4, Loss: 0.035483185201883316, Accuracy: 98.8804931640625, Val Loss: 0.06331119686365128, Val Accuracy: 98.22000122070312
Iteration 3500, Epoch 5, Loss: 0.03611747547984123, Accuracy: 98.7684326171875, Val Loss: 0.07043301314115524, Val Accuracy: 97.97999572753906
Iteration 4200, Epoch 6, Loss: 0.02310101129114628, Accuracy: 99.23754119873047, Val Loss: 0.053132422268390656, Val Accuracy: 98.5999984741211
Iteration 490

Опишите все эксперименты, результаты. Сделайте выводы.

<p>Стандартная архтектура с 3 слоями даёт точность около 97.5% на тестовой выборке.</p>
<p>Добавил ещё несколько свёрточных слоёв, BN после каждого из них. Оптимизатор - Adam, скорость обучения 1e-3 - оптимальная, при изменении точность проседает.</p>

---

<p><b>В итоге точность.</b></p>
<p>На <i>тренеровочной</i> выборке: <i>99.47%</i>.</p>
<p>На <i>валидационной</i> выборке: <i>98.56%</i>.</p>
<p>На <i>тестовой</i> выборке: <i>98.72%</i>.</p>

---

Неплохая точность. Точность в 97% на тестовой выборке достигается очень просто. Усложняя сеть, можно добиться лучшего. Но за эти 1.5-2% плата - увеличение времени обучения более чем в 2 раза.